# Lectura y balanceo de datos

## Lectura

In [20]:
#Paquetes
import tensorflow as tf
import numpy as np

#Extracción de datos
lote = 32 #Numero de muestras con el que se trabaja hasta que se actualizan los parametros
data_training = tf.keras.preprocessing.text_dataset_from_directory( #Se necesita TensorFlow version 2.3.0 o superior
    "aclImdb/train", #Ruta de los datos
    batch_size=lote, #Tamaño del lote
    validation_split=0.2, #Proporcion de muestras de validacion
    subset="training", #Tipo de subset
    seed=1337, #Semilla desde la que empiezo a coger muestras
)
data_validation = tf.keras.preprocessing.text_dataset_from_directory(
    "aclImdb/train",
    batch_size=lote,
    validation_split=0.2,
    subset="validation",
    seed=1337,
)
data_test = tf.keras.preprocessing.text_dataset_from_directory(
   "aclImdb/test", batch_size=lote
)

#Número de lotes
print(
    "Número de lotes de entrenamiento: %d"
    % tf.data.experimental.cardinality(data_training)
)
print(
    "Número de lotes de validación: %d" % tf.data.experimental.cardinality(data_validation)
)
print(
    "Número de lotes de prueba: %d"
    % tf.data.experimental.cardinality(data_test))

In [21]:
#Bucle para extraer los valores
for texto, etiqueta in data_training.take(1):
    for i in range(5):
        print('Texto: ',texto.numpy()[i])
        print('Etiqueta: ',etiqueta.numpy()[i])

# Preparación del texto

In [3]:
#Paquetes
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
import string
import re

# Creamos una función para eliminar las / de nuestro texto, y poner todas las letras en minúsculas
def preparacion(texto):
    texto_minusculas = tf.strings.lower(texto)
    sin_barras = tf.strings.regex_replace(texto_minusculas, "<br />", " ")
    return tf.strings.regex_replace(
        sin_barras, "[%s]" % re.escape(string.punctuation), "")


# Definimos la vectorización.
caracteristicas = 20000 #Número máximo de tokens de la capa (con cuantas palabras trabaja)
long_seq = 500 #Longitud máxima de cada secuencia

# Definimos la capa de vectorizacion
capa_vectorizacion = TextVectorization(
    #Definimos los valores
    standardize=preparacion,
    max_tokens=caracteristicas,
    output_mode="int", #El vector que sale es de enteros
    output_sequence_length=long_seq,
)

#Creamos el vocabulario de nuestra capa de vectorizacion
vocabulario = data_training.map(lambda x, y: x)

#Y lo adaptamos a la capa de vectorizacion
capa_vectorizacion.adapt(vocabulario)

In [4]:
def vectorizacion(texto, etiqueta):
    texto = tf.expand_dims(texto, -1)
    return capa_vectorizacion(texto), etiqueta

# Vectorizamos los datos de entrada.
training = data_training.map(vectorizacion)
validation = data_validation.map(vectorizacion)
test = data_test.map(vectorizacion)

#Mejoramos el rendimiento de nuestra red haciendo un prefetching sobre nuestra memoria caché
training = training.cache().prefetch(buffer_size=10)
validation = validation.cache().prefetch(buffer_size=10)
test = test.cache().prefetch(buffer_size=10)

In [22]:
#Volvemos a extraer los valores con el mismo bucle
for texto, etiqueta in training.take(1):
    for i in range(5):
        print('Texto: ',texto.numpy()[i])
        print('Etiqueta: ',etiqueta.numpy()[i])

## Construccion del modelo

In [8]:
#Métricas (extraidas de codigo de keras)

#Métricas de evaluación
from tensorflow.keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [23]:
#Importamos capas de keras
from tensorflow.keras import layers

# Definimos las entradas del modelo, que van a ser arrays de enteros
entradas = tf.keras.Input(shape=(None,), dtype="int64")

# Despues empleamos una capa de embedding para mapear las caracteristicas extraidas
dimension_embedding = 128
x = layers.Embedding(caracteristicas, dimension_embedding)(entradas)

#Hacemos un dropout para evitar el overfitting
x = layers.Dropout(0.5)(x)

# Ejecutamos 2 capas convolucionales de una dimension para extraer bien la informacion
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x) #Convolucion de filtro de 128 valores, 7 de nucleo y saltos de 3 en 3
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x) #Habilitado el padding y la funcion de activacio ReLU

#Agrupamos los resultados obtenidos
x = layers.GlobalMaxPooling1D()(x)

# Y los sacamos por una densa de 128 neuronas
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)

# Finalmente obtenemos las predicciones con una capa densa de 1 neurona
predicciones = layers.Dense(1, activation="sigmoid", name="predicciones")(x)

model = tf.keras.Model(entradas, predicciones)

# Compile the model with binary crossentropy loss and an adam optimizer.
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy',f1_m,precision_m, recall_m])

#Resumen de la red
model.summary()

In [24]:
#Ajuste del modelo a los datos
historia = model.fit(training, validation_data = validation,
                    epochs=3, #Cuantas iteraciones
                    verbose=True) 

In [25]:
model.evaluate(test)

In [26]:
#Vemos que tenemos recogido en la historia del modelo
historia.history

In [28]:
#Importamos la libreria para pintar 
import matplotlib.pyplot as plt

# Ploteamos como es nuestra exactitud
plt.plot(historia.history['accuracy'])
plt.plot(historia.history['val_accuracy'])
plt.title('Exactitud del modelo')
plt.ylabel('Exactitud')
plt.xlabel('Iteracion')
plt.legend(['Entrenamiento', 'Validacion'], loc='upper left')
plt.show()

# Ploteamos como se minimiza la perdida de nuestro modelo
plt.plot(historia.history['loss'])
plt.plot(historia.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()